In [34]:
# biosignalsnotebooks own package for loading and plotting the acquired data
import biosignalsnotebooks as bsnb

# Scientific packages
from numpy import arange, sin, pi
from numpy.random import randn

In [35]:
# Base packages used in OpenSignals Tools Notebooks for plotting data
from bokeh.plotting import figure, output_file, show, curdoc
from bokeh.io import output_notebook
from bokeh.layouts import column, row
from bokeh.models import ColumnDataSource, Plot, LinearAxis, BoxAnnotation, Arrow, VeeHead, LinearAxis, Range1d
output_notebook(hide_banner=True)

In [36]:
# Function to help us plot the examples
def print_filtfilt_plots(time, signal, lfilter_signal, filtfilt_signal, legend="Original Signal"):
    p = figure(plot_width=400, plot_height=200)
    p.ygrid.grid_line_alpha=0.5
    p.line(time, signal, line_width=1, legend_label=legend)
    p = bsnb.applyOpenSignalsStyle(p)
    p.yaxis.axis_label = 'Value'
    p.xaxis.axis_label = 'time'

    q = figure(plot_width=400, plot_height=200)
    q.ygrid.grid_line_alpha=0.5
    q.line(time, signal, line_width=2, line_color='blue', legend_label=legend, line_alpha=0.5)
    q.line(time, lfilter_signal, line_width=2, line_dash='dashed', line_color='green', legend_label='Filter')
    q = bsnb.applyOpenSignalsStyle(q)
    q.yaxis.axis_label = 'Value'
    q.xaxis.axis_label = 'time'

    r = figure(plot_width=400, plot_height=200)
    r.ygrid.grid_line_alpha=0.5
    r.line(time, signal, line_width=2, line_color='blue', legend_label=legend, line_alpha=0.5)
    r.line(time, filtfilt_signal, line_width=2, line_dash='dashed', line_color='red', legend_label='Filfilt')
    r = bsnb.applyOpenSignalsStyle(r)
    r.yaxis.axis_label = 'Value'
    r.xaxis.axis_label = 'time'
    
    return p,q,r

In [39]:
# Load of data
data, header = bsnb.load("ISB\Filtro de señal ECG\Senales bitalino\Senal_basal_Muñoz.h5", get_header=True)

# The data file is loaded as a Python dictionary, where each key indicates the 
# channel of the acquisition. In the next line we get the channel where the 
# ECG data was acquired.
channel = list(data.keys())[0]

# The header is also in the form of a Python dictionary. The next line saves 
# the sampling frequency of the acquired data
fs = header["sampling rate"]
# Resolution of the sensor (number of available bits)
resolution = header['resolution'][0]

# Signal Samples
signal_raw = data[channel]
time = bsnb.generate_time(signal_raw, fs)

# Let's convert the signal's units, since we know it is a ECG signal
signal = bsnb.raw_to_phy("ECG", "biosignalsplux", signal_raw, resolution, "mV")

# Creating a Butterworth filter with order 3 and low-pass frequency of 300 Hz
lfilter_signal = bsnb.lowpass(signal, 100, order=3)
filtfilt_signal = bsnb.lowpass(signal,100, order=3, use_filtfilt=True)

In [40]:
# Let's plot our example
p,q,r = print_filtfilt_plots(time, signal, lfilter_signal, filtfilt_signal)

p.x_range = Range1d(0, 1)
q.x_range = Range1d(0, 1)
r.x_range = Range1d(0, 1)

show(row(p,q,r))